### 05. använda 'tools' med LLM

- ge en modell tillgång till funktioner för att lösa problem, exempel: 
    - matematiska problem
    - access till privat och/eller ny data
    - utföra operationer i andra system

![05 diagram](docs/05.drawio.png)

In [1]:
import os
from datetime import datetime

from dotenv import load_dotenv

from google import genai
from google.genai import types

load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

client = genai.Client(api_key=API_KEY)

In [2]:
promt = f"What is the current ({datetime.today()}) temperature in Amsterdam?"

In [3]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=promt,
)

print(response.candidates[0].content.parts[0].text)

I am sorry, I cannot give you the exact current temperature in Amsterdam. As a language model, I do not have access to live, real-time weather data.

To get the current temperature, I recommend checking a reliable weather source such as:

*   **A weather website or app:** Popular options include AccuWeather, The Weather Channel, Google Weather, or local Dutch weather services.
*   **A weather API:** If you're a developer, you can use a weather API to get real-time weather data programmatically.

When you check, make sure to specify that you want the temperature for Amsterdam.



In [4]:
# 'tool' som modellen får tillgång till


def get_current_temperature(location: str) -> dict:
    return {"temperature": "10", "unit": "Celcius", "location": location}

In [7]:
# metadata som beskriver 'tool' för modellen

get_current_temperature_tool = {
    "name": "get_current_temperature",
    "description": "returns the current temperature for a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "name of location"},
        },
        "required": ["location"],
    },
}

In [8]:
# anpasa metadata för klienten

temp_tool = types.Tool(function_declarations=[get_current_temperature_tool])
config = types.GenerateContentConfig(tools=[temp_tool])

In [9]:
# när det krävs flera anrop för att lösa en uppgift behövshistoriken

contents = [types.Content(role="user", parts=[types.Part(text=promt)])]

In [10]:
# anropa api

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=contents, config=config
)

# om modellen vill använda 'tool'

if response.candidates[0].content.parts[0].function_call:
    tool_to_call = response.candidates[0].content.parts[0].function_call

    # anropa 'tool'
    if tool_to_call.name == "get_current_temperature":
        tool_result = get_current_temperature(**tool_to_call.args)

        # lägg till modellens svar i chathistoriken
        contents.append(
            types.Content(role="model", parts=[types.Part(function_call=tool_to_call)])
        )

        # lägg till resultatet av 'tool'-anrop till chathistoriken
        tool_result_part = types.Part.from_function_response(
            name=tool_to_call.name, response={"result": tool_result}
        )

        contents.append(types.Content(role="user", parts=[tool_result_part]))

        # anropa api igen med chathistrik som innehåller resultatet av 'tool'-anrop
        response = client.models.generate_content(
            model="gemini-2.0-flash", contents=contents, config=config
        )

        # lägg till modellens svar chathistoriken
        contents.append(
            types.Content(role="model", parts=[types.Part(text=response.text)])
        )

        # skriv ut svar
        print(response.text)

    # skriv ut chathistorik
    for c in contents:
        print(c)

The current temperature in Amsterdam is 10 degrees Celcius.

parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='What is the current (2025-06-05 14:33:37.560749) temperature in Amsterdam?')] role='user'
parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=FunctionCall(id=None, args={'location': 'Amsterdam'}, name='get_current_temperature'), function_response=None, text=None)] role='model'
parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id=None, name='get_current_temperature', response={'result': {'temperature': '10', 'unit': 'Celcius', 'location': 'Amsterdam'}}), text=None)] role='user'
parts=[Par